# Game 3 - Hospital Resources

Skillfully allocating medical resources during a pandemic can save lives. Unexpected and extreme medical events not only disrupt hospitals' normal routines and protocol, but strain resources such as trained staff, space and pharmaceuticals, in addition to PPE and life-saving medical equipment. Determining how to most effectively allocate these resources is paramount to assuring the wellbeing of Sailors far from port.

In this scenario, you will oversee the medical bay of the USS GHOST where COVID-19 has taken hold of the crew. Your task is to create an algorithm to deploy the ship's available resources in a manner that will ensure as many Sailors as possible recover from the virus.

In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations

pd.set_option("display.max_rows", 100)

* +30 10 ventilators (reusable)
* +20 10 oxygen masks (reusable)
    * cannot be combined with ventilator
* +15 10 plasma
* +30 7 remdesivir
* +25 20 Dexamethasone
* +15 10 casirivimab
* +10 17 chloroquine
* 30 beds
    * bed boosts health 5 points
    * bed allocation required for other treatments
* final health score = final health + bed bonus + rate of decline + treatment bonuses
* penalty of -100 if any sailor final health is 0


In [2]:
sailors_df = pd.read_csv("day1data.csv", header=1, index_col=0)
sailors_df2 = pd.read_csv("day2data.csv", header=1, index_col=0)
sailors_df3 = pd.read_csv("day3data.csv", header=1, index_col=0)
resource_list = ["Bed", "Ventilator", "Remdesivir", "Dexamethasone", "Plasma", "Casirivimab", "Supplemental oxygen", "Chloroquine"]
for resource in resource_list:
    sailors_df[resource] = 0
    sailors_df2[resource] = 0
    sailors_df3[resource] = 0
sailors_df["Used"] = ""
sailors_df2["Used"] = ""
sailors_df3["Used"] = ""
columns = ["resource", "count", "amt_used", "health_bonus"]
data = np.array([["Bed", 30, 0, 5],
          ["Ventilator", 10, 0, 30],
          ["Supplemental oxygen", 10, 0, 20],
          ["Plasma", 10, 0, 15],
          ["Remdesivir", 7, 0, 30],
          ["Dexamethasone", 20, 0, 25],
          ["Casirivimab", 10, 0, 15],
          ["Chloroquine", 17, 0, 10]])
resources_df = pd.DataFrame(data=data, columns=columns)
for col in ["count", "amt_used", "health_bonus"]:
    resources_df[col] = pd.to_numeric(resources_df[col])
resources_df["reusable"] = pd.Series([True, True, True, False, False, False, False, False])
print(resources_df,"\n")

              resource  count  amt_used  health_bonus  reusable
0                  Bed     30         0             5      True
1           Ventilator     10         0            30      True
2  Supplemental oxygen     10         0            20      True
3               Plasma     10         0            15     False
4           Remdesivir      7         0            30     False
5        Dexamethasone     20         0            25     False
6          Casirivimab     10         0            15     False
7          Chloroquine     17         0            10     False 



In [3]:
# Reset reusable resources in DataFrame
def reset_reusable(resources_df):
    resources_df.loc[resources_df["reusable"]==True, "count"] += resources_df.loc[resources_df["reusable"]==True, "amt_used"]
    resources_df.loc[resources_df["reusable"]==True, "amt_used"] = 0

def calc_final_score(allocation_df, resources_df):
    final_health = allocation_df["Total"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    resources_df = resources_df[resources_df["reusable"]=="False"]
    resource_bonus = (resources_df["count"]*resources_df["health_bonus"]).sum()/2
    return final_health.sum() + resource_bonus

# Verify that resource counts add up
# Does not verify that treated sailors have beds
def check_dfs(sailors_dfs, resources_df):
    for resource in resources_df.itertuples():
        used = 0
        for df in sailors_dfs:
            if resource.reusable and df[resource.resource].sum() > (resource.count+resource.amt_used):
                print(resource.resource)
                return False
            used += df[resource.resource].sum()
        if not resource.reusable and used != resource.amt_used:
            return False
    return True

In [4]:
def use_resource(sailors_df, resources_df, resource, indices):
    """
    resource = resource name as string
    indices = Pandas index list
    Ex. for single indices do [df.index[position]]
    Ex. for range of values do df.index[start:end]
    """
    bonus_dict = {}
    for r, bonus in zip(resources_df["resource"], resources_df["health_bonus"]):
        bonus_dict[r] = int(bonus)

    sailors_df.loc[indices, resource] += 1
    sailors_df.loc[indices, "Total"] += bonus_dict[resource]
    sailors_df.loc[indices, "Used"] = sailors_df.loc[indices, "Used"].apply(lambda x: ','.join([resource]+x.split(',')))
    resources_df.loc[resources_df["resource"] == resource, "amt_used"] += len(indices)
    resources_df.loc[resources_df["resource"] == resource, "count"] -= len(indices)

def allocate_reusable(sailors_df, resources_df):    
    add_beds = sailors_df.index[:30]
    use_resource(sailors_df, resources_df, "Bed", add_beds)
    
    add_ventilators = sailors_df.index[:10]
    use_resource(sailors_df, resources_df, "Ventilator", add_ventilators)
    
    oxymask_candidates = sailors_df[(sailors_df["Ventilator"]==0) & (sailors_df["Bed"]>0)]
    add_oxymask = oxymask_candidates.index[:10]
    use_resource(sailors_df, resources_df, "Supplemental oxygen", add_oxymask)

def ded(sailors_df, resources_df):
    def get_bonus(tups):
        total = 0
        for name, bonus in tups:
            total += bonus
        return total
    def get_names(tups):
        names = []
        for name, bonus in tups:
            names.append(name)
        return names

    mindict = {}
    minlist = []
    dying = sailors_df.loc[(sailors_df["Total"] <= 0) & (sailors_df["Bed"] > 0)].sort_values(by=["Total"])
    for sailor in dying.itertuples():
        nonreusable = resources_df[(resources_df["reusable"]==False) & (resources_df["count"] > 0)]
        tups = list(zip(nonreusable["resource"], nonreusable["health_bonus"]))
        y = sailor.Total
        comb_length = 1
        while len(minlist) == 0 and comb_length < len(tups):
            for comb in combinations(tups, comb_length):
                x = y + get_bonus(comb)
                mindict[x] = get_names(comb)
            minlist = [k for k in mindict.keys() if k > 0]
            comb_length += 1
        if len(minlist) == 0:
#             print("Sailor with Health:", sailors_df.iloc[2]["Health"], "is dying.")
            continue
        m = min(minlist)
        for resource in mindict[m]:
            use_resource(sailors_df, resources_df, resource, [sailor.Index])
        mindict = {}
        minlist = []

def boosting(sailors_df, resources_df, threshold):
    def get_bonus(tups):
        total = 0
        for name, bonus in tups:
            total += bonus
        return total
    def get_names(tups):
        names = []
        for name, bonus in tups:
            names.append(name)
        return names
    treatable = sailors_df.loc[(sailors_df["Bed"] > 0) & (sailors_df["Total"] > 0)]
    mindict = {}
    minlist = []
    for sailor in treatable.itertuples():
        unavailable = sailor.Used.split(',')
        available = resources_df[(~resources_df["resource"].isin(unavailable)) \
                                 & (resources_df["count"] > threshold) \
                                 & (resources_df["reusable"]==False)]
        
        tups = list(zip(available["resource"], available["health_bonus"]))
        y = sailor.Total
        comb_length = len(tups)
        while len(minlist) == 0 and comb_length > 0:
            for comb in combinations(tups, comb_length):
                x = y + get_bonus(comb)
                mindict[x] = get_names(comb)
            minlist = [k for k in mindict.keys() if k < 101]
            comb_length -= 1
        if len(minlist) == 0:
            continue
        m = min(minlist)
        for resource in mindict[m]:
            
            use_resource(sailors_df, resources_df, resource, [sailor.Index])
        mindict = {}
        minlist = []

def ethical_boosting(sailors_df, resources_df, boost_bool):
    if not boost_bool:
        return
    nonreusable = resources_df.loc[(resources_df["reusable"]==False) & (resources_df["count"] > 0)].copy()
    sailor_idx = 0
    while nonreusable["count"].sum() > 0:
        sailors_df.sort_values(by=["Total"], inplace=True)
        unavailable = sailors_df.loc[sailors_df.index[sailor_idx], "Used"].split(',')
        nonreusable.sort_values(by=["health_bonus"], ascending=False, inplace=True)
        resource_idx = 0
        while resource_idx < len(nonreusable) and nonreusable.loc[nonreusable.index[resource_idx], "resource"] in unavailable:
            resource_idx += 1
        if resource_idx >= len(nonreusable) or sailors_df.loc[sailors_df.index[sailor_idx], "Bed"] == 0:
            sailor_idx += 1
        else:
            r = nonreusable.loc[nonreusable.index[resource_idx], "resource"]
            use_resource(sailors_df, resources_df, r, [sailors_df.index[sailor_idx]])
            nonreusable = resources_df.loc[(resources_df["reusable"]==False) & (resources_df["count"] > 0)].copy()

def allocate(sailors_df, resources_df, threshold):
    resources_df = resources_df.copy()
    sailors_df = sailors_df.copy()
    sailors_df["Total"] = sailors_df["Health"]+sailors_df["Rate of health decline"]
    sailors_df.sort_values(by=["Total"], inplace=True)
    
    allocate_reusable(sailors_df, resources_df)
    sailors_df["After Reusable"] = sailors_df["Total"]
    ded(sailors_df, resources_df)
    ethical_boosting(sailors_df, resources_df, threshold)
    
    return sailors_df, resources_df

def calc_final_score(allocation_df, resources_df):
    final_health = allocation_df["Total"].apply(lambda x: 100 if x > 100 else x)
    final_health = final_health.apply(lambda x: -100 if x < 1 else x)
    resources_df = resources_df[resources_df["reusable"]==False]
    resource_bonus = (resources_df["count"]*resources_df["health_bonus"]).sum()/2
    return final_health.sum() + resource_bonus


In [5]:
a_df, r_df = allocate(sailors_df, resources_df, False)
print(check_dfs([a_df], r_df))

reset_reusable(r_df)
a_df2, r_df2 = allocate(sailors_df2, r_df, False)
print(check_dfs([a_df, a_df2], r_df2))

reset_reusable(r_df2)
a_df3, r_df3 = allocate(sailors_df3, r_df2, True)
print(check_dfs([a_df, a_df2, a_df3], r_df3))

print(calc_final_score(a_df, r_df)+calc_final_score(a_df2, r_df2)+calc_final_score(a_df3, r_df3))
print(r_df)

True
True
True
4726.5
              resource  count  amt_used  health_bonus  reusable
0                  Bed     30         0             5      True
1           Ventilator     10         0            30      True
2  Supplemental oxygen     10         0            20      True
3               Plasma     10         0            15     False
4           Remdesivir      6         1            30     False
5        Dexamethasone     19         1            25     False
6          Casirivimab     10         0            15     False
7          Chloroquine     16         1            10     False


In [6]:
a_df.drop(["Used", "After Reusable"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day1Submission.csv")
a_df2.drop(["Unnamed: 3", "Used", "After Reusable"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day2Submission.csv")
a_df3.drop(["Unnamed: 3", "Used", "After Reusable"], axis=1).sort_values(by="ID").to_csv("Challenge3_Day3Submission.csv")